In [1]:
#cluster a set of documents using Python
#identify the latent structures within the synopses of the top 100 films

#1. tokenizing
#2. stemming (*based on stemming lib, results change)- reduce a word to its stem or root form
#3. calculate cosine distance between each document = measure of similarity
#4. cluster documents using the k-means algorithm
#5. using multidimensional scaling to reduce dimensionality within the corpus
#6. conduct a hierarchical clustering on the corpus using Ward clustering
#7. Latent Dirichlet Allocation(LDA)????

In [2]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
import psycopg2
from IPython.display import display

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
import pymysql

def get_connection():
    conn = psycopg2.connect(host='smith.snu.ac.kr', database='daeb', user='daeb', password='daeb123123', )
    conn.set_client_encoding('UTF8')
    return conn

conn = get_connection()

def get_article_tables():
    curs = conn.cursor()
    
    select_sql = """SELECT id, table_title, strip_tags(CONTENT) as content FROM article_tables order by id""" # limit 10000
    curs.execute(select_sql)
    return curs.fetchall()

In [4]:
#sample data
# from sklearn.datasets import fetch_20newsgroups
# twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
# twenty_train.target_names
# from nltk.corpus import brown
# from keras.datasets import imdb
# (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=1000)


train_data = get_article_tables()

# clean_content = [x['content'].lower() for x in train_data]

In [5]:
len(train_data)

39826

In [6]:
# train_data = pd.DataFrame.from_dict(train_data)
# train_data.head()

In [7]:
rep = {'nbsp':'', 'table':'', 'legend':'', 'mg/dl':'', 'g/l':'', 'yrs':'year', '\n':' ', ';':'', 'kg/m2':'', 'n=':''}#, 'e.g', '(', ')'}

# clean_content = [pattern.sub(lambda m: rep[re.escape(m.group(0))], x['content']) for x in train_data]

rep = dict((re.escape(k), v) for k, v in rep.items())
pattern = re.compile("|".join(rep.keys()))
clean_content = [(pattern.sub(lambda m: rep[re.escape(m.group(0))], x[2])) for x in train_data]

In [8]:
clean_content[100]

' Table 2  Patients’ Characteristics and Hospitalization or Clinic Visits During Follow-Up          Group 1 (15 patients) Group 2 (24 patients) p Value     Age 29 ± 16 25 ± 11 NS   Gender (F/M) 23/2 22/2 NS   Psychiatric diagnosis 100% (15/15) 100% (24/24) NS   Mean duration of symptoms (months) 20 ± 5 22 ± 5 NS   Median monthly number of ER or clinic visits preablation (range) 3 (2–6) 3 (1–5) NS   Median number of ER or clinic admissions 1-month postablation (range) 0 (0–2) 3 (2–5) &lt 0.05   Median number of phone contacts 1-month postablation (range) 4 (2–6) 1 (1–2) &lt 0.05   Median number of phone contacts during the third month postablation (range) 0 (0–1) None &lt 0.05     ER = emergency room.  '

In [9]:
train_data[1000][2]

'\nTable 1\n\nPatient Characteristics Pre-LVAD Implantation\n\n\n\n\n\n\nAge, yrs\n60(51,68)\n\n\nMale\n64(80)\n\n\nBody mass index, kg/m\n2\n\n\n27(24,32)\n\n\nDiabetes mellitus\n22(28)\n\n\nHypertension\n33(41)\n\n\nChronic HF etiology\n\n\n\n\nIdiopathic CMP\n37(46)\n\n\n\nIschemic CMP\n34(43)\n\n\n\nValvular CMP\n5(6)\n\n\n\nChemotherapy CMP\n3(4)\n\n\n\nPeripartum CMP\n1(1)\n\n\nDuration of HF symptoms, yrs\n5.5(1,10)\n\n\nNYHA functional class\n\n\n\n\nIII\n29(36)\n\n\n\nIV\n51(64)\n\n\nLVEF, %\n16(13,23)\n\n\nLVEDD, cm\n6.6(5.8,7.3)\n\n\nCardiac index, l/(min· m\n2\n)\n1.7(1.4,2.0)\n\n\nIABP/Centrimag/ECMO\n8(10)\n\n\nInotrope dependence\n56(70)\n\n\nINTERMACS profile\n\n\n\n\n1\n9(11)\n\n\n\n2\n12(15)\n\n\n\n3\n39(49)\n\n\n\n4\n16(20)\n\n\n\n5\n3(4)\n\n\n\n6\n1(1)\n\n\nLVAD indication\n\n\n\n\nBTT\n52(65)\n\n\n\nBTD\n3(4)\n\n\n\nDT\n25(31)\n\n\n\n\nValues are median (25th, 75th percentiles) or n (%). CentriMag (Ventricular Assist System, Thoratec Corp., Pleasanton, California).

In [10]:
# re.sub('[^A-Za-z ]+', '', train_data[5])
# re.sub('[^A-Za-z0-9 ]+', '', train_data[5])

In [11]:
#stopwords
stopwords = nltk.corpus.stopwords.words('english')
stopwords[:10]

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  Searched in:
    - '/Users/grace/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/Users/grace/workspace/keras/venv/bin/../nltk_data'
    - '/Users/grace/workspace/keras/venv/bin/../share/nltk_data'
    - '/Users/grace/workspace/keras/venv/bin/../lib/nltk_data'
**********************************************************************


In [ ]:
# stemming
# -porter stemmer
# -lancaster stemmer
# -snowball stemmer
from nltk.stem.lancaster import LancasterStemmer
# from nltk.stem.snowball import SnowballStemmer
# stemmer = SnowballStemmer('english')
stemmer = LancasterStemmer()
#tokenizing
def tokenize_and_stem(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #filter tokens not containing letters
    filtered = []
    for token in tokens:
        if re.search('[a-zA-Z]', token) and len(token)>2:
            filtered.append(token)
#     stems = [stemmer.stem(t, pos='v') for t in filtered]
    stems = [stemmer.stem(t) for t in filtered]
    return stems

def tokenize_only(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered = []
    for token in tokens:
        if re.search('[a-zA-Z]', token) and len(token)>2:
            filtered.append(token)
    return filtered

In [ ]:
total_vocab_stemmed = []
total_vocab_tokenized = []

for i in clean_content:
    all_stemmed = tokenize_and_stem(i)
    total_vocab_stemmed.extend(all_stemmed)
    
    all_tokenized = tokenize_only(i)
    total_vocab_tokenized.extend(all_tokenized)

In [ ]:
#create dataframe with stemmed vocab and tokenized words (link)
vocab_frame = pd.DataFrame({'words':total_vocab_tokenized}, index=total_vocab_stemmed)
vocab_frame.drop_duplicates(inplace=True)
vocab_frame.head()

In [ ]:
# len([brown.raw(__id) for __id in [_id for _id in brown.fileids()]])
# data_list = [brown.raw(__id) for __id in [_id for _id in brown.fileids()]]

In [ ]:
#Tf-idf and document similarity
#frequency-inverse document frequencey(tf-idf) vectorize parameters and convert the document list into tf-idf matrix
# 1. count word occurrences by document
# 2. transform into a document-term matrix = term frequency matrix

#max_df = max frequency within the documents
#min_idf = if 5, the term would have to be in at least 5 of the documents to be considered, 0.2 = 20% of documents
#ngram_ranges

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, 
                                   max_features=200000, 
                                   min_df=0.01, 
                                   stop_words='english', 
                                   use_idf=True, 
                                   lowercase=True, 
                                   tokenizer=tokenize_and_stem, ngram_range=(1,2))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(clean_content)
print()
print(tfidf_matrix.shape)

In [ ]:
terms = tfidf_vectorizer.get_feature_names()

In [ ]:
terms

In [ ]:
#dist = cosine similarity of each document
from sklearn.metrics.pairwise import cosine_similarity

dist = 1 - cosine_similarity(tfidf_matrix)

In [ ]:
dist

In [ ]:
#k-means- predetermined number of clusters
# nums = [2,3,4,5,6,7,8,9,10]
nums=[3, 4, 5]
from sklearn.cluster import KMeans
from __future__ import print_function

for num in nums:
    print('cluster : %s' % str(num))
    num_clusters = num
    km = KMeans(n_clusters=num_clusters)
    %time km.fit(tfidf_matrix)
    clusters = km.labels_.tolist()

    documents = {'id':[x[0] for x in train_data],
                'content':clean_content,
                 'title': [x[1] for x in train_data],
                'cluster':clusters}

    clu_docu = pd.DataFrame(documents, index=[clusters], columns=['id','content','title','cluster'])

    print(clu_docu['cluster'].value_counts())

    #top words nearest to the cluster centroid
    print('Top terms per clusters')
    print()
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    for i in range(num_clusters):
        print('Cluster %d words:' % i, end='')

        for ind in order_centroids[i, :]:
            print('%s' % vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0], end=',')
        print()

        print("Cluster %d titles:" % i, end='')
        title_list = clu_docu.loc[i]['title'].values.tolist()
        adver_titles = [x for x in title_list if x.lower().find('adverse')>=0]
        adver_contents = [x for x in clu_docu.loc[i]['content'].values.tolist() if x.lower().find('adverse')>=0]
        print('count of adverse included in title %s' % str(len(adver_titles)))
        print('count of adverse included in content %s' % str(len(adver_contents)))
        print(adver_titles)
#         print(' %s, \n' % title_list[:30])
    #         for title in title_list[:10]:
    #             print(' %s, ' % title, end='')
        print()

In [ ]:
# from sklearn.externals import joblib

# joblib.dump(km, 'doc_cluster.pkl')
# joblib.load('doc_cluster.pkl)

In [ ]:
#multidimensional scaling
import os
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS

mds = MDS(n_components=2, dissimilarity='precomputed', random_state=1)
pos = mds.fit_transform(dist)


In [ ]:
#visualize document clusters
